# [TR-011] Symbolic kinematics

In [ ]:
%%sh
pip install ampform==0.11.4 graphviz==0.17 numpy==1.19.5 qrules==0.9.2 sympy==1.9 > /dev/null

In [ ]:
import inspect
from typing import Any

import black
import graphviz
import numpy as np
import qrules
import sympy as sp
from ampform.data import EventCollection
from ampform.kinematics import _compute_helicity_angles
from ampform.sympy import (
    UnevaluatedExpression,
    create_expression,
    implement_doit_method,
)
from qrules.topology import create_isobar_topologies
from sympy.printing.numpy import NumPyPrinter
from sympy.printing.printer import Printer

This report investigates issue {issue}`56`. The ideal solution would be to use only SymPy in the existing [`ampform.kinematics`](https://ampform.readthedocs.io/en/0.11.4/api/ampform.kinematics.html) module. This has two benefits:

1. It allows computing kinematic variables from four-momenta with different computational back-ends.
2. Expressions for kinematic variable can be inspected through their LaTeX representation.

To simplify things, we investigate 1. by only lambdifying to NumPy. It should be relatively straightforward to lambdify to other back-ends like TensorFlow (as long as they support Einstein summation).

## Test sample

Data sample taken from [this test in AmpForm](https://github.com/ComPWA/ampform/tree/0.11.4/tests/conftest.py#L53-L115) and topology and expected angles taken from [here](https://github.com/ComPWA/ampform/tree/0.11.4/tests/test_kinematics.py#L15-L112).

In [ ]:
topologies = create_isobar_topologies(4)
topology = topologies[1]

In [ ]:
dot = qrules.io.asdot(topology)
graphviz.Source(dot)

In [ ]:
events = EventCollection(
    {
        0: np.array(  # pi0
            [
                (1.35527, 0.514208, -0.184219, 1.23296),
                (0.841933, 0.0727385, -0.0528868, 0.826163),
                (0.550927, -0.162529, 0.29976, -0.411133),
            ]
        ),
        1: np.array(  # gamma
            [
                (0.755744, -0.305812, 0.284, -0.630057),
                (1.02861, 0.784483, 0.614347, -0.255334),
                (0.356875, -0.20767, 0.272796, 0.0990739),
            ]
        ),
        2: np.array(  # pi0
            [
                (0.208274, -0.061663, -0.0211864, 0.144596),
                (0.461193, -0.243319, -0.283044, -0.234866),
                (1.03294, 0.82872, -0.0465425, -0.599834),
            ]
        ),
        3: np.array(  # pi0
            [
                (0.777613, -0.146733, -0.0785946, -0.747499),
                (0.765168, -0.613903, -0.278416, -0.335962),
                (1.15616, -0.458522, -0.526014, 0.911894),
            ]
        ),
    }
)

In [ ]:
angles = _compute_helicity_angles(events, topology)
angles._DataSet__data

## Einstein summation

First challenge is to express the Einstein summation in [the existing implementation](https://github.com/ComPWA/ampform/tree/0.11.4/src/ampform/data.py#L207-L214) in terms of SymPy. The aim is to render the expression resulting nicely as LaTeX while at the same time being able to lambdify the expression to efficient NumPy code. We do this by deriving from [`UnevaluatedExpression`](https://ampform.readthedocs.io/en/0.11.4/api/ampform.sympy.html#ampform.sympy.UnevaluatedExpression) and using the decorator functions provided by the [`ampform.sympy`](https://ampform.readthedocs.io/en/0.11.4/api/ampform.sympy.html) module.

### Define boost and rotation classes

First, wrap rotations and boosts in a class so with a nice LaTeX printer. {ref}`Later on <report/011:Define lambdification>`, a NumPy printer method will be defined externally for each of them.

In [ ]:
@implement_doit_method()
class BoostZ(UnevaluatedExpression):
    def __new__(cls, beta: sp.Symbol, **hints: Any) -> "BoostZ":
        return create_expression(cls, beta, **hints)

    def evaluate(self) -> sp.Expr:
        gamma = 1 / sp.sqrt(1 - beta ** 2)
        return sp.Matrix(
            [
                [gamma, 0, 0, -gamma * beta],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [-gamma * beta, 0, 0, gamma],
            ]
        )

    def _latex(self, printer, *args) -> str:
        beta, *_ = self.args
        beta = printer._print(beta)
        return fR"\boldsymbol{{B_z}}\left({beta}\right)"

In [ ]:
@implement_doit_method()
class RotationY(UnevaluatedExpression):
    def __new__(cls, angle: sp.Symbol, **hints: Any) -> "RotationY":
        return create_expression(cls, angle, **hints)

    def evaluate(self) -> sp.Expr:
        angle = self.args[0]
        return sp.Matrix(
            [
                [1, 0, 0, 0],
                [0, sp.cos(angle), 0, sp.sin(angle)],
                [0, 0, 1, 0],
                [0, -sp.sin(angle), 0, sp.cos(angle)],
            ]
        )

    def _latex(self, printer, *args) -> str:
        angle, *_ = self.args
        angle = printer._print(angle)
        return fR"\boldsymbol{{R_y}}\left({angle}\right)"

In [ ]:
@implement_doit_method()
class RotationZ(UnevaluatedExpression):
    def __new__(cls, angle: sp.Symbol, **hints: Any) -> "RotationZ":
        return create_expression(cls, angle, **hints)

    def evaluate(self) -> sp.Expr:
        angle = self.args[0]
        return sp.Matrix(
            [
                [1, 0, 0, 0],
                [0, sp.cos(angle), -sp.sin(angle), 0],
                [0, sp.sin(angle), sp.cos(angle), 0],
                [0, 0, 0, 1],
            ]
        )

    def _latex(self, printer, *args) -> str:
        angle, *_ = self.args
        angle = printer._print(angle)
        return fR"\boldsymbol{{R_y}}\left({angle}\right)"

### Define Einstein summation class

Similarly, we define a `ArrayMultiplication` class that will eventually be lambdified to `np.einsum`.

In [ ]:
@implement_doit_method()
class ArrayMultiplication(UnevaluatedExpression):
    def __new__(cls, *tensors: sp.Symbol, **hints: Any):
        return create_expression(cls, *tensors, **hints)

    def evaluate(self) -> sp.Expr:
        tensors = self.args
        return sp.Mul(*tensors)

    def _latex(self, printer, *args) -> str:
        tensors_latex = map(printer._print, self.args)
        return " ".join(tensors_latex)

Indeed an expression involving these classes looks nice on the top-level:

In [ ]:
n_events = 3
# n_events = sp.Symbol("n", integer=True, positive=True)
momentum = sp.MatrixSymbol("p", m=n_events, n=4)
beta = sp.Symbol("beta")
phi = sp.Symbol("phi")
theta = sp.Symbol("theta")

boosted_momentum = ArrayMultiplication(
    BoostZ(beta),
    RotationY(-theta),
    RotationZ(-phi),
    momentum,
)
boosted_momentum

The expression can be further evaluated:

In [ ]:
boosted_momentum.doit()

And can even be expressed in terms of matrix elements:

In [ ]:
boosted_momentum.doit().as_explicit()

:::{note}

It could be that the above can be achieved with SymPy's `ArrayTensorProduct` and `ArrayContraction`. See for instance {issue}`sympy/sympy#22279`.

:::

### Define lambdification

In [ ]:
# small helper function
def print_lambdify(symbols, expr):
    np_expr = sp.lambdify(symbols, expr)
    src = inspect.getsource(np_expr)
    src = black.format_str(src, mode=black.Mode(line_length=79))
    print(src)

Now we have a problem: lambdification does not work...

In [ ]:
print_lambdify([beta, theta, momentum], boosted_momentum)

But lambdification can be defined externally to both the SymPy library and the expression classes. Here's an implementation for NumPy where we define the lambdification **through the expression class** (with a `_numpycode` method):

In [ ]:
def print_as_numpy(self, printer: Printer, *args: Any) -> str:
    def multiply(matrix, vector):
        return (
            'np.einsum("ij...,j...",'
            f" np.transpose({matrix}, axes=(1, 2, 0)),"
            f" np.transpose({vector}))"
        )

    def recursive_multiply(tensors):
        if len(tensors) < 2:
            raise ValueError("Need at least two tensors")
        if len(tensors) == 2:
            return multiply(tensors[0], tensors[1])
        return multiply(tensors[0], recursive_multiply(tensors[1:]))

    tensors = list(map(printer._print, self.args))
    if len(tensors) == 0:
        return ""
    if len(tensors) == 1:
        return tensors[0]
    return recursive_multiply(tensors)


ArrayMultiplication._numpycode = print_as_numpy

In [ ]:
print_lambdify(
    symbols=[beta, theta, momentum],
    expr=ArrayMultiplication(beta, theta, momentum),
)

This also needs to be done for the rotation and boost classes:

In [ ]:
print_lambdify([beta, theta, momentum], boosted_momentum)

This time, we define the lambdification **through the printer class**:

In [ ]:
def _print_BoostZ(self: NumPyPrinter, expr: BoostZ) -> str:
    self.module_imports["numpy"].update({"array", "ones", "zeros", "sqrt"})
    arg = expr.args[0]
    beta = self._print(arg)
    gamma = f"1 / sqrt(1 - {beta} ** 2)"
    n_events = f"len({beta})"
    zeros = f"zeros({n_events})"
    ones = f"ones({n_events})"
    return f"""array(
        [
            [{gamma}, {zeros}, {zeros}, -{gamma} * {beta}],
            [{zeros}, {ones}, {zeros}, {zeros}],
            [{zeros}, {zeros}, {ones}, {zeros}],
            [-{gamma} * {beta}, {zeros}, {zeros}, {gamma}],
        ]
    ).transpose(2, 0, 1)"""


NumPyPrinter._print_BoostZ = _print_BoostZ

In [ ]:
def _print_RotationY(self: NumPyPrinter, expr: RotationY) -> str:
    self.module_imports["numpy"].update(
        {"array", "cos", "ones", "zeros", "sin"}
    )
    arg = expr.args[0]
    angle = self._print(arg)
    n_events = f"len({angle})"
    zeros = f"zeros({n_events})"
    ones = f"ones({n_events})"
    return f"""array(
        [
            [{ones}, {zeros}, {zeros}, {zeros}],
            [{zeros}, cos({angle}), {zeros}, sin({angle})],
            [{zeros}, {zeros}, {ones}, {zeros}],
            [{zeros}, -sin({angle}), {zeros}, cos({angle})],
        ]
    ).transpose(2, 0, 1)"""


NumPyPrinter._print_RotationY = _print_RotationY

In [ ]:
def _print_RotationZ(self: NumPyPrinter, expr: RotationZ) -> str:
    self.module_imports["numpy"].update(
        {"array", "cos", "ones", "zeros", "sin"}
    )
    arg = expr.args[0]
    angle = self._print(arg)
    n_events = f"len({angle})"
    zeros = f"zeros({n_events})"
    ones = f"ones({n_events})"
    return f"""array(
        [
            [{ones}, {zeros}, {zeros}, {zeros}],
            [{zeros}, cos({angle}), -sin({angle}), {zeros}],
            [{zeros}, sin({angle}), cos({angle}), {zeros}],
            [{zeros}, {zeros}, {zeros}, {ones}],
        ]
    ).transpose(2, 0, 1)"""


NumPyPrinter._print_RotationZ = _print_RotationZ

In [ ]:
print_lambdify([beta, theta, momentum], boosted_momentum)

:::{note}

The code above contains a lot of duplicate code, such as `len(-phi)`. This could possibly be improved with {class}`~sympy.codegen.ast.CodeBlock`. See {issue}`ComPWA/ampform#166`.

:::